# DX 704 Week 9 Project

This week's project will build an email spam classifier based on the Enron email data set.
You will perform your own feature extraction, and use naive Bayes to estimate the probability that a particular email is spam or not.
Finally, you will review the tradeoffs from different thresholds for automatically sending emails to the junk folder.

The full project description and a template notebook are available on GitHub: [Project 9 Materials](https://github.com/bu-cds-dx704/dx704-project-09).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download Data Set

We will be using the Enron spam data set as prepared in this GitHub repository.

https://github.com/MWiechmann/enron_spam_data

You may need to download this differently depending on your environment.

In [1]:
!wget https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip

zsh:1: command not found: wget


In [16]:
import pandas as pd
import numpy as np

In [4]:
# pandas can read the zip file directly
enron_spam_data = pd.read_csv("enron_spam_data.zip")
enron_spam_data

,Message ID,Subject,Message,Spam/Ham,Date
0,0,christmas tree farm pictures,NaN,ham,1999-12-10
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,1999-12-13
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,1999-12-14
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,1999-12-14
4,4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham,1999-12-14
...,...,...,...,...,...
33711,33711,= ? iso - 8859 - 1 ? q ? good _ news _ c = eda...,"hello , welcome to gigapharm onlinne shop .\np...",spam,2005-07-29
33712,33712,all prescript medicines are on special . to be...,i got it earlier than expected and it was wrap...,spam,2005-07-29
33713,33713,the next generation online pharmacy .,are you ready to rock on ? let the man in you ...,spam,2005-07-30
33714,33714,bloow in 5 - 10 times the time,learn how to last 5 - 10 times longer in\nbed ...,spam,2005-07-30


In [5]:
(enron_spam_data["Spam/Ham"] == "spam").mean()

np.float64(0.5092834262664611)

## Part 2: Design a Feature Extractor

Design a feature extractor for this data set and write out two files of features based on the text.
Don't forget that both the Subject and Message columns are relevant sources of text data.
For each email, you should count the number of repetitions of each feature present.
The auto-grader will assume that you are using a multinomial distribution in the following problems.

In [26]:
# YOUR CHANGES HERE
import re
import json

def extract_features(text):
    if pd.isna(text):
        return {}

    text = text.lower()
    text = re.sub(r"[^a-z0-9]+", " ", text) 

    tokens = text.split()

    features = {}
    for tok in tokens:
        features[tok] = features.get(tok, 0) + 1

    return features

feature_rows = []
for _, row in enron_spam_data.iterrows():
    msg_id = row["Message ID"]
    text = str(row["Subject"]) + " " + str(row["Message"])
    feats = extract_features(text)

    feature_rows.append({
        "Message ID": msg_id,
        "features_json": json.dumps(feats)
    })

feature_df = pd.DataFrame(feature_rows)

Assign a row to the test data set if `Message ID % 30 == 0` and assign it to the training data set otherwise.
Write two files, "train-features.tsv" and "test-features.tsv" with two columns, Message ID and features_json.
The features_json column should contain a JSON dictionary where the keys are your feature names and the values are integer feature values.
This will give us a sparse feature representation.


In [27]:
# YOUR CHANGES HERE

train_df = feature_df[enron_spam_data["Message ID"] % 30 != 0]
test_df  = feature_df[enron_spam_data["Message ID"] % 30 == 0]

train_df.to_csv("train-features.tsv", sep="\t", index=False)
test_df.to_csv("test-features.tsv", sep="\t", index=False)

train_df.head(), test_df.head()

(   Message ID                                      features_json
 1           1  {"vastar": 6, "resources": 4, "inc": 4, "gary"...
 2           2  {"calpine": 2, "daily": 2, "gas": 2, "nominati...
 3           3  {"re": 2, "issue": 4, "fyi": 1, "see": 1, "not...
 4           4  {"meter": 3, "7268": 3, "nov": 3, "allocation"...
 5           5  {"mcmullen": 2, "gas": 5, "for": 2, "11": 1, "...,
      Message ID                                      features_json
 0             0  {"christmas": 1, "tree": 1, "farm": 1, "pictur...
 30           30  {"january": 3, "production": 3, "estimate": 5,...
 60           60  {"new": 2, "pooling": 2, "point": 2, "7342": 1...
 90           90  {"enron": 3, "actuals": 1, "for": 1, "january"...
 120         120  {"re": 2, "revision": 5, "black": 4, "marlin":...)

Submit "train-features.tsv" and "test-features.tsv" in Gradescope.

Hint: these features will be graded based on the test accuracy of a logistic regression based on the training features.
This is to make sure that your feature set is not degenerate; you do not need to compute this regression yourself.
You can separately assess your feature quality based on your results in part 6.

## Part 3: Compute Conditional Probabilities

Based on your training data, compute appropriate conditional probabilities for use with naïve Bayes.
Use of additive smoothing with $\alpha=1$ to avoid zeros.


In [28]:
# YOUR CHANGES HERE
from collections import defaultdict

train_feats = pd.read_csv("train-features.tsv", sep="\t")

train = train_feats.merge(enron_spam_data[["Message ID", "Spam/Ham"]], on="Message ID")

ham_counts = defaultdict(int)
spam_counts = defaultdict(int)
total_ham = 0
total_spam = 0

for _, row in train.iterrows():
    feats = json.loads(row["features_json"])
    if row["Spam/Ham"] == "ham":
        for word, cnt in feats.items():
            ham_counts[word] += cnt
            total_ham += cnt
    else:
        for word, cnt in feats.items():
            spam_counts[word] += cnt
            total_spam += cnt

vocab = set(ham_counts.keys()) | set(spam_counts.keys())
V = len(vocab)
alpha = 1 

rows = []
for word in vocab:
    ham_prob = (ham_counts[word] + alpha) / (total_ham + alpha*V)
    spam_prob = (spam_counts[word] + alpha) / (total_spam + alpha*V)
    rows.append({"feature": word, "ham_probability": ham_prob, "spam_probability": spam_prob})

df = pd.DataFrame(rows)

Save the conditional probabilities in a file "feature-probabilities.tsv" with columns feature, ham_probability and spam_probability.

In [29]:
# YOUR CHANGES HERE

df.to_csv("feature-probabilities.tsv", sep="\t", index=False)

df.head()

,feature,ham_probability,spam_probability
0,sancho,2.187633e-07,2.493528e-06
1,gsbpidl,2.187633e-07,8.311760e-07
2,gesture,2.843923e-06,2.493528e-06
3,weasterl,4.375266e-07,2.770587e-07
4,offically,4.375266e-07,2.770587e-07


Submit "feature-probabilities.tsv" in Gradescope.

## Part 4: Implement a Naïve Bayes Classifier

Implement a naïve Bayes classifier based on your previous feature probabilities.

In [30]:
# YOUR CHANGES HERE

probs = pd.read_csv("feature-probabilities.tsv", sep="\t")

train = train_feats.merge(enron_spam_data[["Message ID", "Spam/Ham"]], on="Message ID")

p_ham = (train["Spam/Ham"] == "ham").mean()
p_spam = (train["Spam/Ham"] == "spam").mean()

log_prior_ham = np.log(p_ham)
log_prior_spam = np.log(p_spam)

ham_lookup = dict(zip(probs.feature, probs.ham_probability))
spam_lookup = dict(zip(probs.feature, probs.spam_probability))

def predict(features_json):
    feats = json.loads(features_json)

    log_ham = log_prior_ham
    log_spam = log_prior_spam

    for word, count in feats.items():
        if word in ham_lookup:
            log_ham += count * np.log(ham_lookup[word])
        if word in spam_lookup:
            log_spam += count * np.log(spam_lookup[word])

    max_log = max(log_ham, log_spam)
    ham_score = np.exp(log_ham - max_log)
    spam_score = np.exp(log_spam - max_log)

    total = ham_score + spam_score

    return ham_score / total, spam_score / total

pred_rows = []
for _, row in train.iterrows():
    ham_prob, spam_prob = predict(row["features_json"])
    pred_rows.append({
        "Message ID": row["Message ID"],
        "ham": ham_prob,
        "spam": spam_prob
    })

pred_df = pd.DataFrame(pred_rows)


Save your prediction probabilities to "train-predictions.tsv" with columns Message ID, ham and spam.

In [31]:
# YOUR CHANGES HERE

pred_df.to_csv("train-predictions.tsv", sep="\t", index=False)

pred_df.head()

,Message ID,ham,spam
0,1,1.0,9.632258e-187
1,2,1.0,1.490928e-12
2,3,1.0,6.205975e-155
3,4,1.0,2.247626e-150
4,5,1.0,4.886654e-40


Submit "train-predictions.tsv" in Gradescope.

## Part 5: Predict Spam Probability for Test Data

Use your previous classifier to predict spam probability for the test data.

In [32]:
# YOUR CHANGES HERE

test_feats = pd.read_csv("test-features.tsv", sep="\t")

p_ham = (train["Spam/Ham"] == "ham").mean()
p_spam = (train["Spam/Ham"] == "spam").mean()

log_prior_ham = np.log(p_ham)
log_prior_spam = np.log(p_spam)

ham_lookup = dict(zip(probs.feature, probs.ham_probability))
spam_lookup = dict(zip(probs.feature, probs.spam_probability))

def predict(features_json):
    feats = json.loads(features_json)

    log_ham = log_prior_ham
    log_spam = log_prior_spam

    for word, count in feats.items():
        if word in ham_lookup:
            log_ham += count * np.log(ham_lookup[word])
        if word in spam_lookup:
            log_spam += count * np.log(spam_lookup[word])

    m = max(log_ham, log_spam)
    ham_s = np.exp(log_ham - m)
    spam_s = np.exp(log_spam - m)
    total = ham_s + spam_s

    return ham_s / total, spam_s / total

rows = []
for _, row in test_feats.iterrows():
    ham_prob, spam_prob = predict(row["features_json"])
    rows.append({
        "Message ID": row["Message ID"],
        "ham": ham_prob,
        "spam": spam_prob
    })

Save your prediction probabilities in "test-predictions.tsv" with the same columns as "train-predictions.tsv".

In [33]:
# YOUR CHANGES HERE

test_pred_df = pd.DataFrame(rows)
test_pred_df.to_csv("test-predictions.tsv", sep="\t", index=False)

test_pred_df.head()

,Message ID,ham,spam
0,0,0.054955,9.450447e-01
1,30,1.000000,8.622434e-85
2,60,1.000000,1.115310e-12
3,90,1.000000,7.227356e-34
4,120,1.000000,1.836670e-188


Submit "test-predictions.tsv" in Gradescope.

## Part 6: Construct ROC Curve

For every probability threshold from 0.01 to .99 in increments of 0.01, compute the false and true positive rates from the test data using the spam class for positives.
That is, if the predicted spam probability is greater than or equal to the threshold, predict spam.

In [34]:
# YOUR CHANGES HERE

test_pred = pd.read_csv("test-predictions.tsv", sep="\t")
test = test_pred.merge(enron_spam_data[["Message ID", "Spam/Ham"]], on="Message ID")
test["is_spam"] = (test["Spam/Ham"] == "spam").astype(int)

thresholds = np.arange(0.01, 1.00, 0.01)

rows = []
for thresh in thresholds:
    test["pred_spam"] = (test["spam"] >= thresh).astype(int)
    
    TP = ((test["pred_spam"] == 1) & (test["is_spam"] == 1)).sum()
    FP = ((test["pred_spam"] == 1) & (test["is_spam"] == 0)).sum()
    FN = ((test["pred_spam"] == 0) & (test["is_spam"] == 1)).sum()
    TN = ((test["pred_spam"] == 0) & (test["is_spam"] == 0)).sum()
    
    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0  
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0  
    
    rows.append({
        "threshold": round(thresh, 2),
        "false_positive_rate": FPR,
        "true_positive_rate": TPR
    })

Save this data in a file "roc.tsv" with columns threshold, false_positive_rate and true_positive rate.

In [35]:
# YOUR CHANGES HERE

roc_df = pd.DataFrame(rows)
roc_df.to_csv("roc.tsv", sep="\t", index=False)

roc_df.head()

,threshold,false_positive_rate,true_positive_rate
0,0.01,0.034420,0.996503
1,0.02,0.034420,0.996503
2,0.03,0.032609,0.996503
3,0.04,0.030797,0.996503
4,0.05,0.028986,0.996503


Submit "roc.tsv" in Gradescope.

## Part 7: Signup for Gemini API Key

Create a free Gemini API key at https://aistudio.google.com/app/api-keys.
You will need to do this with a personal Google account - it will not work with your BU Google account.
This will not incur any charges unless you configure billing information for the key.

You will be asked to start a Gemini free trial for week 11.
This will not incur any charges unless you exceed expected usage by an order of magnitude.


No submission needed.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

Used re and json libraries for text/string manipulation and matching.